# **Análisis y Limpieza de Datos de Ventas**

### **DATASET:**
Se tiene un conjunto de datos que representa las ventas diarias de una tienda en línea. Este conjunto contiene información sobre los productos vendidos, la cantidad, el precio por unidad, y la fecha de venta. Sin embargo, algunos de los datos están incompletos o tienen valores nulos. 

### **OBJETIVOS:**
El objetivo es limpiar y analizar los datos, y calcular algunas medidas clave para entender mejor las ventas.
* Manipular arrays con numpy: Para operaciones matemáticas básicas con los datos.
* Manejar datos con pandas: Para cargar el dataset, analizar su estructura y calcular medidas de tendencia central.
* Trabajar con archivos JSON: Para guardar y leer la información transformada.
* Manejar datos nulos: Para identificar y lidiar con valores faltantes.
* Aplicar ciclos, condicionales y operadores lógicos/matemáticos: Para realizar cálculos y aplicar reglas de negocio.

In [ ]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import os
import json

In [ ]:
# Cargar el archivo CSV
file_path = 'clases/ds-fundamentals/data/ejercicios_integracion/ventas_tienda_simulada.csv'
df = pd.read_csv(file_path)

# Mostrar los primeros registros del dataframe para visualizar los datos
print("Datos originales:")
display(df)

In [ ]:
# Identificar datos nulos
print("\nDatos nulos por columna:")
print(df.isnull().sum())

# Manejo de datos nulos:
# 1. Rellenar 'cantidad' y 'precio_unitario' con la media de la columna.
df['cantidad'].fillna(df['cantidad'].mean(), inplace=True)
df['precio_unitario'].fillna(df['precio_unitario'].mean(), inplace=True)

# 2. Calcular el valor de 'total' faltante como cantidad * precio_unitario
df['total'] = df['total'].fillna(df['cantidad'] * df['precio_unitario'])

# Verificar los datos luego del manejo de nulos
print("\nDatos luego de manejar valores nulos:")
display(df)

In [ ]:
# Guardar el dataframe limpio en un archivo JSON
output_file_path = os.path.join(os.path.dirname(file_path), 'ventas_tienda_simulada_limpio.json')
df.to_json(output_file_path, orient='records', lines=True)

# Leer el archivo JSON para verificar que se haya guardado correctamente
with open(output_file_path, 'r') as f:
    data_json = f.readlines()

print(f"\nArchivo JSON guardado en: {output_file_path}")
print("\nPrimeros 3 registros en el archivo JSON:")
print(data_json[:3])

In [ ]:
# Medidas de tendencia central usando numpy
cantidad_array = np.array(df['cantidad'])
precio_array = np.array(df['precio_unitario'])
total_array = np.array(df['total'])

print("\nMedidas de tendencia central:")
print(f"Media de la cantidad vendida: {np.mean(cantidad_array)}")
print(f"Media del precio unitario: {np.mean(precio_array)}")
print(f"Media del total de ventas: {np.mean(total_array)}")

# Estadísticas descriptivas generales usando pandas
print("\nEstadísticas descriptivas:")
print(df.describe())
